## PySM 3 dust templates based on Planck GNILC maps

The purpose of this notebook is to pre-process galactic dust maps from the [Planck analysis with GNILC](https://arxiv.org/abs/1605.09387)
to create a dust model for PySM 3 which is based on real data at large scale and has added gaussian fluctuations at small scales.

These dust maps, compared to the commander results used by the `d1` model of PySM 2, have the CIB signal removed and less noise.
This notebook focuses only on the IQU templates at 353 GHz, "dust temperature" and "dust spectral index" will be processed separately.

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "64" # for jupyter.nersc.gov otherwise the notebook only uses 2 cores

In [ ]:
import pymaster as nmt

In [ ]:
import os
import healpy as hp
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
%matplotlib inline

In [ ]:
hp.disable_warnings()

In [ ]:
plt.style.use("seaborn-talk")

In [ ]:
import pysm3 as pysm
import pysm3.units as u

In [ ]:
nside = 2048
lmax = 2048

In [ ]:
comp = "IQU"

In [ ]:
components = list(enumerate(comp))
components

In [ ]:
spectra_components = ["TT", "EE", "BB", "TE"]

## Mask

In [ ]:
planck_mask = hp.reorder(
    fits.open("data/HFI_Mask_GalPlane-apo0_2048_R2.00.fits")[1].data["GAL060"], n2r=True)

In [ ]:
np.unique(planck_mask) # no apodization

In [ ]:
hp.mollview(planck_mask, title=f"Planck common galactic mask, {comp}")

In [ ]:
(hp.nside2resol(nside) * u.radian).to(u.arcmin)

In [ ]:
mask_apo = nmt.mask_apodization(planck_mask, (30*u.arcmin).to_value(u.degree))

In [ ]:
(mask_apo==1).sum()/len(mask_apo) * 100

In [ ]:
(mask_apo==0).sum()/len(mask_apo) * 100

In [ ]:
hp.mollview(mask_apo)

In [ ]:
hp.mollview(mask_apo - planck_mask)

In [ ]:
planck_mask = mask_apo

## Planck GNILC dust

Downloaded from the Planck Legacy Archive (PLA), the [GNILC templates](https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/Foreground_maps#GNILC_thermal_dust_maps) are available either at a single resolution of 80 arcminutes or at variable resolution, where regions of higher emission have 5 arcmin resolution. Here we want to fit the slope of the small scales therefore we want the variable resolution image which has the least smoothing applied.

Rename `varres` to `unires` in the filename to use the 80 arcmin uniform resolution map instead.

In [ ]:
dust_map_filename = "COM_CompMap_IQU-thermaldust-gnilc-varres_2048_R3.00.fits"
dust_map_path = os.path.join("data", dust_map_filename)

In [ ]:
if not os.path.exists(dust_map_path):
    !wget -O $dust_map_path http://pla.esac.esa.int/pla/aio/product-action?MAP.MAP_ID=$dust_map_filename

In [ ]:
fits.open(dust_map_path).info()

In [ ]:
fits.open(dust_map_path)[1].data.dtype

In [ ]:
II_cov = hp.read_map(dust_map_path, ["II_COV"])

In [ ]:
hp.mollview(II_cov, norm="log")

In [ ]:
m_planck,h = hp.read_map(dust_map_path, [c+"_STOKES" for c in comp], h=True)

In [ ]:
#h

Input units are $K_{CMB}$, PySM templates use $\mu K_{RJ}$

In [ ]:
m_planck <<= u.K_CMB

In [ ]:
m_planck = m_planck.to("uK_RJ", equivalencies=u.cmb_equivalencies(353*u.GHz)) 

In [ ]:
if m_planck.ndim == 1:
    m_planck = m_planck.reshape((1,-1))

In [ ]:
m_planck[0] = (hp.read_map("data/COM_CompMap_Dust-GNILC-F353_2048_R2.00.fits") * u.MJy/u.sr).to(
    "uK_RJ", equivalencies=u.cmb_equivalencies(353*u.GHz)
)

In [ ]:
(np.isinf(m_planck[0])).sum()

In [ ]:
for i_pol, pol in components:
    hp.mollview(m_planck[i_pol], title="Planck-GNILC dust " + pol, unit=m_planck.unit, min=-300, max=300)

## Build the log of the polarization tensor

I - PxP and polarization fraction

  $
    \left[ \begin{array}{cc} i+q & u \\ u & i-q \end{array} \right] \ = \
    \ln \left[ \begin{array}{cc} I+Q & U \\ U & I-Q \end{array} \right]
  $


$
    i = \frac{1}{2}\, \ln \left(I^2-P^2\right), \hspace{1em}
    q = \frac{1}{2}\, \frac{Q}{P}\, \ln \frac{I+P}{I-P}, \hspace{1em}
    u = \frac{1}{2}\, \frac{U}{P}\, \ln \frac{I+P}{I-P}
$

$    I = e^{i} \cosh p, \hspace{2em}
    Q = \frac{q}{p} e^{i} \sinh p, \hspace{2em}
    U = \frac{u}{p} e^{i} \sinh p
$


In [ ]:
from log_pol_tens_utils import map_to_log_pol_tens, log_pol_tens_to_map

In [ ]:
log_pol_tens = map_to_log_pol_tens(m_planck.value)

In [ ]:
m_back = log_pol_tens_to_map(log_pol_tens)

In [ ]:
m_back - m_planck.value

In [ ]:
hp.mollview((m_back - m_planck.value)[1])

In [ ]:
del m_back

In [ ]:
for i in range(3):
    log_pol_tens[i, np.isnan(log_pol_tens[i])] = np.nanmedian(log_pol_tens[i])

In [ ]:
assert np.isnan(log_pol_tens).sum() == 0

In [ ]:
hp.mollview(log_pol_tens[0], title="Log pol tensor I", max=6)

In [ ]:
log_pol_tens = hp.ma(log_pol_tens)

In [ ]:
log_pol_tens.mask = planck_mask==0

In [ ]:
np.linalg.norm(hp.fit_dipole(log_pol_tens[0])[1])

In [ ]:
mono, dip_vector = hp.fit_dipole(log_pol_tens[0])

In [ ]:
log_pol_tens[0]

In [ ]:
np.linalg.norm(dip_vector)

In [ ]:
def remove_dip_inplace(m, dip_vector):
    npix = m.shape[-1]
    nside = hp.npix2nside(npix)
    m.data[:] -= np.dot(dip_vector, hp.pix2vec(nside, np.arange(npix)))

In [ ]:
remove_dip_inplace(log_pol_tens[0], dip_vector)

In [ ]:
log_pol_tens[0].data[:] -= mono

In [ ]:
np.linalg.norm(hp.fit_dipole(log_pol_tens[0])[1])

In [ ]:
hp.mollview(log_pol_tens[0], title="Log pol tensor I")

In [ ]:
hp.mollview(log_pol_tens.data[0], title="Log pol tensor I")

In [ ]:
hp.mollview(log_pol_tens[1], title="Log pol tensor Q")

In [ ]:
hp.mollview(log_pol_tens[2], title="Log pol tensor U")

## Angular power spectrum with `namaster`

We use `namaster` to estimate the power spectrum of the masked map,
compared to `anafast`, `namaster` properly deconvolves the mask to remove the
correlations between different $C_\ell$ caused by the mask.

We don't need to deconvolve the beam, we won't be using the values at high $\ell$ anyway.

In [ ]:
from utils import run_namaster
ell, cl_norm, cl = run_namaster(log_pol_tens, planck_mask, lmax)

The maps are smoothed (with different beams in different regions), so we try to select the region of $\ell$ which is roughtly linear in `loglog` scale just before the curvature of the smoothing looks to start dominating. We will use this region to fit to extrapolate small scale power to high $\ell$.

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
def model(ell, A, gamma):
    out = A * ell ** gamma
    return out

In [ ]:
ell_fit_low = {"II":900, "EE":150, "BB":150, "TE":150}
ell_fit_high = {"II":1400, "EE":400, "BB":400, "TE":400}

In [ ]:
    plt.figure()
    plt.loglog(ell, ell*(ell+1)/np.pi/2 * cl[pol], label="NaMaster $C_\ell$")
    # plt.semilogx(ell, ell*(ell+1)/np.pi/2 * cl[pol], label="NaMaster $C_\ell smoothed$")
    # plt.loglog(ell, ell*(ell+1)/np.pi/2 * cl_sigma_G[pol], label="SS $C_\ell$")

    plt.plot(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], 
             model(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], A_fit[pol], gamma_fit[pol]), label="model fit")
    
    #plt.plot(w_ell**2,  label=f"Beam {c} $C_\ell$")

    plt.axvline(ell_fit_low[pol], linestyle="--", color="black", label="$ \ell={} $".format(ell_fit_low[pol]))
    plt.axvline(ell_fit_high[pol], linestyle="--", color="gray", label="$ \ell={} $".format(ell_fit_high[pol]))
    plt.legend()
    plt.grid()
    plt.title(f"{pol} power spectrum for dust")
    plt.ylabel("$\ell(\ell+1)C_\ell/2\pi [\mu K_{RJ}]$")
    plt.xlabel(("$\ell$"))
    #plt.xlim(0, 400)
    #plt.ylim(1, 30);
    #plt.xlim(ell_fit_low[pol]//2, lmax)
    #plt.ylim(0, 1e-2 if pol == "II" else 1e-3)

In [ ]:
A_fit, gamma_fit, A_fit_std, gamma_fit_std = {},{},{},{}
for pol in spectra_components:
    xdata = np.arange(ell_fit_low[pol], ell_fit_high[pol])
    ydata = np.fabs(xdata*(xdata+1)/np.pi/2 * cl[pol][xdata])
    (A_fit[pol], gamma_fit[pol]), cov = curve_fit(model, xdata, ydata)

    A_fit_std[pol], gamma_fit_std[pol] = np.sqrt(np.diag(cov))
    plt.figure()
    plt.loglog(ell, np.fabs(ell*(ell+1)/np.pi/2 * cl[pol]), label="NaMaster $C_\ell$")
    # plt.semilogx(ell, ell*(ell+1)/np.pi/2 * cl[pol], label="NaMaster $C_\ell smoothed$")
    # plt.loglog(ell, ell*(ell+1)/np.pi/2 * cl_sigma_G[pol], label="SS $C_\ell$")

    plt.plot(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], 
             model(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], A_fit[pol], gamma_fit[pol]), label="model fit")
    
    #plt.plot(w_ell**2,  label=f"Beam {c} $C_\ell$")

    plt.axvline(ell_fit_low[pol], linestyle="--", color="black", label="$ \ell={} $".format(ell_fit_low[pol]))
    plt.axvline(ell_fit_high[pol], linestyle="--", color="gray", label="$ \ell={} $".format(ell_fit_high[pol]))
    plt.legend()
    plt.grid()
    plt.title(f"{pol} power spectrum for dust")
    plt.ylabel("$\ell(\ell+1)C_\ell/2\pi [\mu K_{RJ}]$")
    plt.xlabel(("$\ell$"))
    #plt.xlim(0, 400)
    #plt.ylim(1, 30);
    plt.xlim(ell_fit_low[pol]//2, lmax)
    plt.ylim(0, 1e-2 if pol == "II" else 1e-3)

In [ ]:
A_fit, A_fit_std

In [ ]:
gamma_fit, gamma_fit_std

### Transition

Instead of clipping the measured spectra at some $\ell$ and stitching the small scale extrapolation, I use a sigmoid function to make a sharp transition between the 2 curves

In [ ]:
from log_pol_tens_utils import sigmoid

In [ ]:
def sigmoid(x, x0, width, power=3):
      """Sigmoid function given start point and width
  
      Parameters
      ----------
      x : array
          input x axis
      x0 : float
          value of x where the sigmoid starts (not the center)
      width : float
          width of the transition region in unit of x
      power : float
          tweak the steepness of the curve
  
      Returns
      -------
      sigmoid : array
          sigmoid, same length of x"""
        
      return 1.0 / (1 + np.exp(-power * (x - x0) / width)) 
      #fwhm = 20 * u.arcmin
      #return 1 - hp.gauss_beam(fwhm=fwhm.to_value(u.radian), lmax=x[-1])**2
      #return np.heaviside(x - x0, 0)

In [ ]:
for pol in ["II", "EE"]:
    plt.figure()
    plt.loglog(sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10), label="sigmoid")
    plt.loglog(1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10), label="1-sigmoid")
    plt.loglog(np.sqrt(1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10)), label="sqrt 1-sigmoid")
    plt.axvline(ell_fit_high[pol], color="black", label=f"$\ell={ell_fit_high[pol]}$")
    plt.axvline(ell_fit_high[pol] + ell_fit_high[pol]/10, color="gray", label=f"$\ell={ell_fit_high[pol]+ ell_fit_high[pol]//10}$")
    plt.ylim(1e-5, 2)
    plt.grid()
    plt.legend();
    plt.title(f"Sigmoid {pol}")
    plt.xlim(ell_fit_low[pol], ell_fit_high[pol]*1.5);

## Simulate a realization of small scales fluctuations

We simulate small scales fluctuations using the inverse of the window function and the fitted amplitude and slope.
Also fix the seed to make this reproducible.

In [ ]:
np.random.seed(777)

In [ ]:
zeros = np.zeros(len(ell), dtype=np.double)

In [ ]:
output_nside = 8192
output_lmax = output_nside * 2
output_ell = np.arange(output_lmax+1)
output_cl_norm = output_ell * (output_ell+1) / 2 / np.pi

In [ ]:
#small_scales_input_cl = [
#   np.heaviside(output_ell - ell_fit_low[pol]/2, 1) * model(output_ell, A_fit[pol], gamma_fit[pol]) * sigmoid(output_ell, ell_fit_high[pol], ell_fit_high[pol]/10) / output_cl_norm for pol in spectra_components
#]

In [ ]:
small_scales_input_cl = np.array([
   model(output_ell, A_fit[pol], gamma_fit[pol]) * sigmoid(output_ell, ell_fit_high[pol], ell_fit_high[pol]/10) / output_cl_norm for pol in spectra_components
])

In [ ]:
small_scales_input_cl[:, 0] = 0

In [ ]:
assert np.isnan(small_scales_input_cl).sum() == 0

In [ ]:
assert np.isinf(small_scales_input_cl).sum() == 0

In [ ]:
hp.write_cl("data/cl_dust_log_pol_tens_small_scales_neworder.fits", small_scales_input_cl, overwrite=True)

In [ ]:
small_scales_input_cl = np.array([
    #np.heaviside(ell - ell_fit_low[pol]/2, 1)  \
    1 \
    * model(ell, A_fit[pol], gamma_fit[pol]) \
    * sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10)  \
    / cl_norm \
    for pol in spectra_components
])

In [ ]:
small_scales_input_cl[:, 0] = 0

In [ ]:
for i in range(4):
    plt.loglog(small_scales_input_cl[i], label=str(i))
plt.legend()
plt.grid();

In [ ]:
m_sigma_G = hp.synfast(np.vstack([small_scales_input_cl, [zeros, zeros]]), nside, new=True)

In [ ]:
hp.mollview(m_sigma_G[0])

In [ ]:
hp.mollview(m_sigma_G[1])

In [ ]:
temp_cl = hp.anafast(m_sigma_G, lmax=lmax, use_pixel_weights=True)
pol = "II"
plt.loglog(temp_cl[0]*cl_norm)
plt.loglog(small_scales_input_cl[0]*cl_norm)
plt.loglog(model(ell, A_fit[pol], gamma_fit[pol]))
plt.grid();
#del temp_cl;

## Remove small scales from the input maps

In [ ]:
alm_log_pol_tens_fullsky = hp.map2alm(log_pol_tens.data, lmax=lmax, use_pixel_weights=True)

In [ ]:
hp.mollview(log_pol_tens.data[0])

In [ ]:
pol="II"
hp.almxfl(alm_log_pol_tens_fullsky[0],np.sqrt(1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10)), inplace=True)

In [ ]:
pol = "EE"
sigmoid_pol = np.sqrt(1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10))
for i in [1,2]:
    hp.almxfl(alm_log_pol_tens_fullsky[i], sigmoid_pol, inplace=True)

In [ ]:
hp.write_alm("data/alm_dust_log_pol_tens_large_scales.fits", alm_log_pol_tens_fullsky, overwrite=True)

In [ ]:
log_pol_tens_largescales = hp.alm2map(alm_log_pol_tens_fullsky, nside=nside)

In [ ]:
log_pol_tens_largescales = hp.ma(log_pol_tens_largescales)

In [ ]:
log_pol_tens_largescales.mask = log_pol_tens.mask

In [ ]:
mono, dip = hp.fit_dipole(log_pol_tens_largescales[0])

In [ ]:
remove_dip_inplace(log_pol_tens_largescales[0], dip)
log_pol_tens_largescales[0].data[:] -= mono

In [ ]:
hp.mollview(log_pol_tens_largescales[0])

In [ ]:
ell, cl_norm, cl_log_pol_tens_largescales = run_namaster(log_pol_tens_largescales, planck_mask, lmax=lmax)

In [ ]:
pol = "II"
plt.loglog(cl_log_pol_tens_largescales["II"] * cl_norm, label="NaMaster II masked log pol tensor")
plt.plot(ell, 
             model(ell, A_fit[pol], gamma_fit[pol]) \
             * (1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10)), label="model fit with sigmoid cut")
plt.grid()
plt.legend()
plt.xlim(1000,lmax)

## Combine input maps without small scales to the simulated gaussian fluctuations

In [ ]:
output_log_pol_tens_map = log_pol_tens_largescales + m_sigma_G

In [ ]:
hp.mollview(log_pol_tens_largescales[1])

In [ ]:
hp.mollview(m_sigma_G[0])

In [ ]:
output_map = log_pol_tens_to_map(output_log_pol_tens_map)

In [ ]:
hp.write_map(f"data/gnilc_plus_smallscales_{comp}_nside{nside}.fits", output_map, overwrite=True)

## Compare spectra

In [ ]:
from utils import run_namaster

In [ ]:
output_log_pol_tens_map = hp.ma(output_log_pol_tens_map)
output_log_pol_tens_map.mask = log_pol_tens.mask

In [ ]:
output_log_pol_tens_map[0] = hp.remove_dipole(output_log_pol_tens_map[0])

In [ ]:
_, _, cl_output_log_pol_tens = run_namaster(output_log_pol_tens_map, planck_mask, lmax=lmax)

In [ ]:
_, _, namaster_small_scales_cl = run_namaster(m_sigma_G, planck_mask, lmax=lmax)

In [ ]:
small_scales_cl = hp.anafast(m_sigma_G[0], lmax=lmax, use_pixel_weights=True)

In [ ]:
small_scales_cl = hp.anafast(m_sigma_G[0], lmax=lmax, use_pixel_weights=True)

In [ ]:
A_fit, gamma_fit, A_fit_std, gamma_fit_std = {},{},{},{}
for i_pol, pol in enumerate(spectra_components):
    xdata = np.arange(ell_fit_low[pol], ell_fit_high[pol])
    ydata = xdata*(xdata+1)/np.pi/2 * cl[pol][xdata]
    (A_fit[pol], gamma_fit[pol]), cov = curve_fit(model, xdata, ydata)

    A_fit_std[pol], gamma_fit_std[pol] = np.sqrt(np.diag(cov))
    plt.figure()
    plt.loglog(ell, cl_norm * cl[pol], label="NaMaster $C_\ell$")
    plt.loglog(ell, cl_norm * cl_output_log_pol_tens[pol][:len(ell)], label="Output $C_\ell$")
    # plt.loglog(ell, cl_norm * small_scales_cl, label="small scales $C_\ell$")
    #plt.loglog(ell, cl_norm * namaster_small_scales_cl["II"], label="small scales $C_\ell$")

    plt.plot(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], 
             model(ell[ell_fit_low[pol]//2:ell_fit_high[pol]*2], A_fit[pol], gamma_fit[pol]), label="model fit")
    
    plt.plot(small_scales_input_cl[i_pol]*cl_norm, label="Small scales $C_\ell$")
    #plt.plot(cl_log_pol_tens_largescales["II"]*cl_norm, label="Large scales $C_\ell$")
    
    plt.axvline(ell_fit_low[pol], linestyle="--", color="black", label="$ \ell={} $".format(ell_fit_low[pol]))
    plt.axvline(ell_fit_high[pol], linestyle="--", color="gray", label="$ \ell={} $".format(ell_fit_high[pol]))
    #plt.loglog(1e-6*(1-sigmoid(ell, ell_fit_high[pol], ell_fit_high[pol]/10))*cl_norm, label="1-sigmoid")

    plt.legend()
    plt.grid()
    plt.title(f"{pol} power spectrum for dust")
    plt.ylabel("$\ell(\ell+1)C_\ell/2\pi [\mu K_{RJ}]$")
    plt.xlabel(("$\ell$"))
    #plt.xlim(0, 400)
    #plt.ylim(1, 30);
    plt.xlim(ell_fit_low[pol]*.9, lmax)
    plt.ylim(1e-5 if pol == "II" else 1e-6, 1e-2 if pol == "II" else 1e-3)